In [1]:
from transformers import BertTokenizer, BertModel
import torch

# Carga el tokenizador y el modelo BETO
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model_beto = BertModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

# Oración de ejemplo
sentence = "Hola, esto es una prueba con BERT."

# Tokeniza la oración
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Obtiene los embeddings
with torch.no_grad():
    outputs = model_beto(**inputs)

# Los embeddings de la última capa oculta serían
last_hidden_states = outputs.last_hidden_state

# Embedding del [CLS] token que representa la oración entera
sentence_embedding = last_hidden_states[:, 0, :]

# Embeddings de cada token/palabra en la oración
word_embeddings = last_hidden_states

print("Embedding de la oración:", sentence_embedding)
print("Embeddings de las palabras:", word_embeddings)


/home/spoturno/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/spoturno/.local/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding de la oración: tensor([[ 6.3808e-01, -2.7869e-01, -1.9846e-01, -1.4776e-01,  3.0291e-01,
          7.8052e-01,  9.3867e-02,  7.2238e-01, -4.6559e-02,  4.8533e-01,
          1.1943e-01, -1.0878e+00,  3.5126e-01, -3.5981e-01,  4.9066e-01,
         -7.4640e-01, -4.0938e-01, -1.9730e-01, -3.7721e-02,  1.0112e-01,
          1.4318e-01,  5.2502e-02,  6.6922e-02, -1.2755e-01, -1.3663e-01,
          5.8162e-03,  3.3418e-01, -8.0998e-02, -4.7620e-01, -8.3185e-02,
          1.0710e-01,  1.7079e-01, -2.2541e-01,  4.2820e-01,  5.9104e-01,
         -4.4022e-01,  1.2989e-02, -3.1058e-01, -3.8460e-01, -6.4842e-01,
          1.3952e-01,  8.7414e-01,  7.6276e-01,  6.0126e-01, -3.9091e-02,
          1.7234e-01, -1.6539e-01,  1.2365e-01,  1.5036e-01, -5.5027e-01,
          3.2842e-01, -2.8708e-02, -3.6269e-01, -2.3796e-01,  6.1217e-01,
         -1.7270e-02,  2.6563e-01,  1.4247e-01, -4.9820e-01,  5.1619e-01,
          7.6938e-01,  2.9431e-01,  5.6767e-02, -3.4350e-01,  2.2901e-01,
          5.4

In [2]:
last_hidden_states.shape

torch.Size([1, 13, 768])

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/data_mlsp_fing.csv', encoding = "utf8")

In [4]:
df.head()

,oracion,palabraCompleja,evaluacion_normalizada
0,trabajo abnegado de los sanitarios,abnegado,3.0
1,El impacto de la X Solidaria es un ejemplo par...,paradigmático,4.0
2,Es además una enfermedad que afecta a todos lo...,incidencia,1.5
3,Es la primera vez que me toca participar en un...,autoridades,3.0
4,"Es probable, por lo tanto, y al igual que ocur...",seno,2.0


In [5]:
#rename columns
df.rename(columns={'oracion': 'sentence', 'palabraCompleja': 'target_word', 'evaluacion_normalizada': 'complexity'}, inplace=True)
df.head()

,sentence,target_word,complexity
0,trabajo abnegado de los sanitarios,abnegado,3.0
1,El impacto de la X Solidaria es un ejemplo par...,paradigmático,4.0
2,Es además una enfermedad que afecta a todos lo...,incidencia,1.5
3,Es la primera vez que me toca participar en un...,autoridades,3.0
4,"Es probable, por lo tanto, y al igual que ocur...",seno,2.0


In [6]:
#normalizemos la columna de complexity dividiendo entre 5 todo
df['complexity'] = df['complexity']/5

In [7]:
df.head()

,sentence,target_word,complexity
0,trabajo abnegado de los sanitarios,abnegado,0.6
1,El impacto de la X Solidaria es un ejemplo par...,paradigmático,0.8
2,Es además una enfermedad que afecta a todos lo...,incidencia,0.3
3,Es la primera vez que me toca participar en un...,autoridades,0.6
4,"Es probable, por lo tanto, y al igual que ocur...",seno,0.4


In [8]:
torch.cuda.device_count()

1

In [9]:
# check for cuda device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [10]:
import torch

device = torch.device("cpu")

model_beto.to(device)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Suponiendo que df es tu DataFrame que incluye las columnas 'sentence', 'target_word' y 'complexity'
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

from tqdm.auto import tqdm

def get_embeddings(text_list, tokenizer, model, device):
    """Obtiene embeddings [CLS] para una lista de textos, con barra de progreso."""
    model.eval()
    embeddings = []
    for text in tqdm(text_list, desc="Processing", leave=True):
        encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        with torch.no_grad():
            output = model(**encoded_input)
        embeddings.append(output.last_hidden_state[:, 0, :].squeeze().cpu())
    return torch.stack(embeddings)


def process_dataframe(df, tokenizer, model, device):
    """Procesa un dataframe para obtener embeddings y complejidades."""
    sentence_embeddings = get_embeddings(df['sentence'].tolist(), tokenizer, model, device)
    word_embeddings = get_embeddings(df['target_word'].tolist(), tokenizer, model, device)
    complexities = torch.tensor(df['complexity'].values, dtype=torch.float).unsqueeze(1)
    return torch.cat((sentence_embeddings, word_embeddings, complexities), dim=1)

# Procesar los conjuntos de entrenamiento y prueba
train_data = process_dataframe(train_df, tokenizer, model_beto, device)
test_data = process_dataframe(test_df, tokenizer, model_beto, device)

Processing: 100%|██████████| 465/465 [00:14<00:00, 32.59it/s]


In [12]:
train_data.shape, test_data.shape

(torch.Size([1859, 1537]), torch.Size([465, 1537]))

In [13]:
# Entradas de entrenamiento (todas las columnas excepto la última)
train_features = train_data[:, :-1]
# Objetivos de entrenamiento (última columna)
train_targets = train_data[:, -1]

# Entradas de prueba
test_features = test_data[:, :-1]
# Objetivos de prueba
test_targets = test_data[:, -1]

# Asegúrate de que los tensores estén en la CPU para convertirlos a arrays de NumPy
train_features_np = train_features.numpy()
train_targets_np = train_targets.numpy()
test_features_np = test_features.numpy()
test_targets_np = test_targets.numpy()


In [14]:
import tensorflow as tf

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

2024-03-21 18:55:40.598908: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 18:55:40.599167: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 18:55:40.601370: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 18:55:40.626575: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 18:55:41.158180: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [15]:
import tensorflow as tf

# Definir el modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(train_features.shape[1],)),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=rmse,
              metrics=['mean_absolute_error'])

model.summary()

/home/spoturno/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │       393,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 426,497 (1.63 MB)

 Trainable params: 426,497 (1.63 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Entrenamiento del modelo
history = model.fit(train_features_np, train_targets_np,
                    validation_split=0.2,
                    epochs=5,
                    batch_size=32)

Epoch 1/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5558 - mean_absolute_error: 0.4920 - val_loss: 0.2487 - val_mean_absolute_error: 0.1996
Epoch 2/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2410 - mean_absolute_error: 0.1978 - val_loss: 0.2424 - val_mean_absolute_error: 0.1969
Epoch 3/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2172 - mean_absolute_error: 0.1732 - val_loss: 0.2415 - val_mean_absolute_error: 0.1980
Epoch 4/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2022 - mean_absolute_error: 0.1622 - val_loss: 0.2517 - val_mean_absolute_error: 0.2005
Epoch 5/5
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2026 - mean_absolute_error: 0.1612 - val_loss: 0.2687 - val_mean_absolute_error: 0.2054


In [18]:
# Evaluación del modelo
test_loss, test_mae = model.evaluate(test_features_np, test_targets_np, verbose=2)
print(f"Test RMSE: {test_loss}, Test MAE: {test_mae}")

15/15 - 0s - 1ms/step - loss: 0.2433 - mean_absolute_error: 0.1923
Test RMSE: 0.24329641461372375, Test MAE: 0.19230590760707855


In [53]:
model.save("modelos/Beto.keras")


In [19]:
import csv
import re
import pandas as pd

input_file_path = '/home/spoturno/coding/MLSP_Data/Data/Trial/Spanish/multilex_trial_es_lcp.tsv'
output_file_path = input_file_path.replace('.tsv', '_results.csv')

df = pd.read_csv(input_file_path, delimiter='\t', header=None, names=['en', 'english', 'sentence', 'target_word', 'complexity'])

df.head()


,en,english,sentence,target_word,complexity
0,es_1,spanish,Notifique a su Banco o institución financiera ...,notifique,0.450
1,es_2,spanish,Una falta de liquidez ocasiona problemas que p...,sueldos,0.225
2,es_3,spanish,Una falta de liquidez ocasiona problemas que p...,amenazar,0.575
3,es_4,spanish,Una falta de liquidez ocasiona problemas que p...,ocasiona,0.575
4,es_5,spanish,Cuando se visita a los norteamericanos y se es...,norteamericanos,0.150


In [21]:
trial_set = process_dataframe(df, tokenizer, model_beto, device)
trial_set = trial_set[:, :-1]

Processing: 100%|██████████| 30/30 [00:00<00:00, 35.11it/s]


In [22]:
prediction = model.predict(trial_set.numpy())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [23]:
prediction

array([[0.31075418],
       [0.1860368 ],
       [0.2870497 ],
       [0.16819888],
       [0.313484  ],
       [0.4292397 ],
       [0.40438223],
       [0.36227912],
       [0.40441304],
       [0.33298883],
       [0.289402  ],
       [0.2841792 ],
       [0.35151643],
       [0.2934751 ],
       [0.20819339],
       [0.2374098 ],
       [0.292369  ],
       [0.32097536],
       [0.26388097],
       [0.2576243 ],
       [0.17913207],
       [0.37311977],
       [0.10950947],
       [0.28669083],
       [0.3322345 ],
       [0.3000164 ],
       [0.55594456],
       [0.41000155],
       [0.19340724],
       [0.4658618 ]], dtype=float32)

In [24]:
df['complexity'] = prediction

In [25]:
df

,en,english,sentence,target_word,complexity
0,es_1,spanish,Notifique a su Banco o institución financiera ...,notifique,0.310754
1,es_2,spanish,Una falta de liquidez ocasiona problemas que p...,sueldos,0.186037
2,es_3,spanish,Una falta de liquidez ocasiona problemas que p...,amenazar,0.287050
3,es_4,spanish,Una falta de liquidez ocasiona problemas que p...,ocasiona,0.168199
4,es_5,spanish,Cuando se visita a los norteamericanos y se es...,norteamericanos,0.313484
5,es_6,spanish,Cuando se visita a los norteamericanos y se es...,desvíos,0.429240
6,es_7,spanish,Cuando se visita a los norteamericanos y se es...,legistas,0.404382
7,es_8,spanish,Dicha prosperidad o seguridad está estrechamen...,sustituto,0.362279
8,es_9,spanish,Dicha prosperidad o seguridad está estrechamen...,vinculada,0.404413
9,es_10,spanish,Dicha prosperidad o seguridad está estrechamen...,dicha,0.332989


In [27]:
df.to_csv('/home/spoturno/coding/MLSP/metrics/beto_with_mlp/predictiones.csv', index=False)

In [29]:
import numpy as np
np.sqrt(0.0622)

0.2493992782667985